# Lesson 11: groupby

최규빈  
2023-07-25

<a href="https://colab.research.google.com/github/guebin/PP2023SUM/blob/main/posts/Day2/04_Function, Iteration/ls11.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 강의영상

<https://youtu.be/XlwSIZPcBpM?list=PLQqh36zP38-y8RXbbZBO1POaMqaZPTPCI>

# imports

In [24]:
import numpy as np
import pandas as pd

# df.groupby

`-` 예제1: 아래의 예제에서 그룹별 평균을 구하여라.

In [26]:
_category = ['A']*5+['B']*5
_value = np.concatenate([np.random.randn(5), np.random.randn(5)+10])
df = pd.DataFrame({'category':_category, 'value':_value})
df

`-` 방법1: groupby를 이용하지 않는 방법들

In [45]:
# 이렇게 해도 되고 
df[df.category == 'A'].value.mean(), df[df.category == 'B'].value.mean()

In [44]:
# 이것도 가능
{s:df[df.category == s].value.mean() for s in set(df.category)}

`-` 방법2: groupby만 사용

In [49]:
grouped_list = list(df.groupby('category'))
grouped_list

In [55]:
bundle1, bundle2 = grouped_list

In [60]:
group_name1, sub_df1 = bundle1

In [64]:
display(sub_df1)

In [73]:
for group_name, sub_df in df.groupby('category'):
    print(group_name)
    display(sub_df)

A
B

In [77]:
{group_name:sub_df.value.mean() for group_name, sub_df in df.groupby('category')}

`-` 방법3: groupby().aggregate()

In [85]:
grouped = df.groupby('category')
grouped.aggregate(np.mean)

In [88]:
df.groupby(['category']).aggregate(np.mean)

***groupby(?)에서 올 수 있는 구조***

-   열의이름
-   \[열의이름,열의이름\]

***aggregate(?)에서 올 수 있는 구조***

-   함수: 함수자체[1]가 오거나, 함수를 의미하는 문자열[2] 이 올 수 있음.
-   리스트: \[함수, 함수\] \# 여기에서 함수자리에는 함수자체, 혹은
    함수문자열 아무것이나 올 수 있음.
-   딕셔너리1: {열의이름:함수}
-   딕셔너리2: {열의이름:\[함수,함수\]}

[1] np.mean,sum

[2] ‘size’, ‘count’, ‘sum’, ‘mean’, ‘median’, ‘min’, ‘max’, ‘std’, ‘var’

In [122]:
df.groupby(['category']).aggregate({'value':[np.mean,'size']})

# pd.cut + df.groupby

`-` 구간별 count

In [123]:
arr = np.random.rand(1000)

In [126]:
_bin = [0, 0.2, 0.5, 0.9, 1]
_bin

`-` \[0,0.2,0.5,0.9,1\] 구간에 몇개의 숫자들이 있는지 알고 싶고, 구간별
평균도 알고싶다.

In [140]:
pd.DataFrame({'cat':pd.cut(arr,_bin), 'arr':arr}).groupby('cat').aggregate({'arr':['size',np.mean]})

# flights data

In [148]:
df=pd.read_csv('https://raw.githubusercontent.com/PacktPublishing/Pandas-Cookbook/master/data/flights.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58492 entries, 0 to 58491
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   MONTH      58492 non-null  int64  
 1   DAY        58492 non-null  int64  
 2   WEEKDAY    58492 non-null  int64  
 3   AIRLINE    58492 non-null  object 
 4   ORG_AIR    58492 non-null  object 
 5   DEST_AIR   58492 non-null  object 
 6   SCHED_DEP  58492 non-null  int64  
 7   DEP_DELAY  57659 non-null  float64
 8   AIR_TIME   57474 non-null  float64
 9   DIST       58492 non-null  int64  
 10  SCHED_ARR  58492 non-null  int64  
 11  ARR_DELAY  57474 non-null  float64
 12  DIVERTED   58492 non-null  int64  
 13  CANCELLED  58492 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 6.2+ MB

`-` 예제1: 항공사(AIRLINE)별로 도착지연시간의(ARR_DELAY)의 평균을
구하라.

In [156]:
df.groupby('AIRLINE').aggregate({'ARR_DELAY':'mean'})

`-` 예제2: 항공사(AIRLINE)별로 비행취소건수(CANCELLED)의 합계를 구하라.
취소건수가 가장 높은 두개의 항공사는 어디인가?

In [159]:
df.groupby('AIRLINE').aggregate({'CANCELLED':'sum'})

`-` 예제3: 항공사(AIRLINE)별로 비행취소율(CANCELLED)을 구하라.
비행취소율이 가장 높은 두개의 항공사는 어디인가?

In [162]:
df.groupby('AIRLINE').aggregate({'CANCELLED':'mean'})
# df.groupby('AIRLINE').aggregate({'CANCELLED':lambda x : sum(x)/len(x)})

`-` 예제4: 비행취소율이 가장 높은 두개의 항공사(AIRLINE)를 선택하라. 그
두 항공사에 대하여 요일별(WEEKDAY) 비행취소율(CANCELLED)을 조사하라.

In [167]:
df.groupby('AIRLINE').aggregate({'CANCELLED':'mean'})

MQ와 EV

`-` 예제5: 아래는 운행거리의 요약통계량이다.

In [172]:
df.DIST.describe()

운행거리를 구간별로 `[-np.inf,391,690,1199,np.inf]`와 같이 나눈뒤
비행취소건수와 취소율을 구하여라.

In [183]:
_bin = [-np.inf,391,690,1199,np.inf]
df.assign(DIST2 = pd.cut(df.DIST,_bin)).groupby('DIST2').aggregate({'CANCELLED':['mean','sum']})

# Quiz: HRDataset_v14 자료분석

아래의 코드를 활용하여 Kaggle의
[HRdataset](https://www.kaggle.com/datasets/rhuebner/human-resources-data-set)을
불러오라.

In [8]:
df = pd.read_csv('https://raw.githubusercontent.com/guebin/DV2022/master/posts/HRDataset_v14.csv')
df

다음을 읽고 참 거짓을 판단하여라.

`1`. 근무인원수가 가장 많은 인종(RaceDesc)은 ’White’이며 이는
’Asian’인종과 ’Black or African American’의 합보다 많다.

`2`. ’RaceDesc==White’의 성별(Sex)임금차이는 2000이상이다.

`3`. 퇴직한사람(Termd==1)은 모두 104명이며 백인여성의 퇴직자수가 가장
많다.

`4`. 퇴직한사람중 아시아인의 비율은 10%가 넘지 않는다.

아래를 조사하라.

`5`. \[성별(Sex), 결혼유무(MarriedID)\] 별 연봉(Salary)의 평균을
조사하라. 어떠한 그룹이 가장 평균연봉이 적은가?

`6`. \[성별(Sex), 인종별(RaceDesc)\] 별 연봉(Salary)의
중앙값(‘median’)을 조사하라. 연봉의 중앙값이 가장 높은 그룹은 무엇인가?

`7`. \[성별(Sex), 인종별(RaceDesc)\] 별 연봉(Salary)의
중앙값(‘median’)과 근무인원수를 함께 조사하라. 연봉의 중앙값이 가장 높은
그룹이 혜택을 받는 그룹이라고 느껴지는가?

In [184]:
!git add . 
!git commit -m .
!git push
!quarto publish --no-browser --no-prompt

[main 7780095] .
 2 files changed, 1186 insertions(+), 6 deletions(-)
Enumerating objects: 13, done.
Counting objects: 100% (13/13), done.
Delta compression using up to 16 threads
Compressing objects: 100% (7/7), done.
Writing objects: 100% (7/7), 2.07 KiB | 2.07 MiB/s, done.
Total 7 (delta 4), reused 0 (delta 0)
remote: Resolving deltas: 100% (4/4), completed with 4 local objects.
To https://github.com/guebin/PP2023SUM.git
   a8c13c7..7780095  main -> main
From https://github.com/guebin/PP2023SUM
 * branch            gh-pages   -> FETCH_HEAD
Rendering for publish:

[ 1/27] 1_1.qmd
[ 2/27] about.qmd
[ 3/27] 1_2.qmd
[ 4/27] 2_2.qmd
[ 5/27] posts/Day3/08_PandasBackend/ls21.ipynb
[ 6/27] posts/Day3/08_PandasBackend/ls20.ipynb
[ 7/27] posts/Day3/07_ggplot2/ls16.ipynb
[ 8/27] posts/Day3/07_ggplot2/ls19.ipynb
[ 9/27] posts/Day3/07_ggplot2/ls18.ipynb
[10/27] posts/Day3/07_ggplot2/ls17.ipynb
[11/27] posts/Day2/03_Pandas/ls07.ipynb
[12/27] posts/Day2/03_Pandas/ls08.ipynb
[13/27] posts/Day2/03_P